# CVRP

In a capacitated vehicle routing problem (CVRP), we are given a set of locations $N = \{ 0, ..., n-1 \}$ where $0$ is the depot and $\{ 1, ..., n-1 \}$ are customers.
We need to pick up commodities from the customers using $m$ vehicles with capacity $q$, which start from and return to the depot.
By visiting customer $i$, the load of vehicle increases by weight $d_i$.
Visiting customer $j$ from $i$ incurs the travel cost $c_{ij}$.
The objective is to find a set of tours for the vehicles that visit all customers while minimizing the total travel cost.

## DP Formulation

Consider constructing tours for the vehicles one by one.
Let $k$ be the number of used vehicles (including the current vehicle), $U$ be the set of unvisited customers, $i$ be the current location of the vehicle, and $l$ be the load of the vehicle.
Customer $j$ can be visited by the current vehicle if $l + d_i \leq q$.
Otherwise, we need to return to the depot and use a new vehicle to visit $j$, which is possible only if $k < m$.
Let $V(U, i, l, k)$ be the minimum cost to visit customers $U$ from $i$ with the load $l$ using $m - k + 1$ vehicles.
We have the following DP model:

$$
\begin{align}
    \text{compute } & V(N \setminus \{ 0 \}, 0, 0, 1) \\
    & V(U, i, l, k) = \begin{cases}
        \min\left\{ \min\limits_{j \in U : l + d_i \leq q} c_{ij} + V(U \setminus \{ j \}, j, l + d_i, k), \min\limits_{j \in U} c_{i0} + c_{0j} + V(U \setminus \{ j \}, j, d_i, k + 1) \right\} & \text{if } k < m \\
        \min\limits_{j \in U : l + d_i \leq q} c_{ij} + V(U \setminus \{ j \}, j, l + d_i, k) & \text{if } k = m \\
        c_{i0} + V(U, 0, k, l) & \text{if } U = \emptyset \land i \neq 0 \\
        0 & \text{if } U = \emptyset \land i = 0.
    \end{cases}
\end{align}
$$

When two states $(U, i, l, k)$ and $(U, i, l', k')$ have the same set of unvisited customers $U$ and the same location $i$, if $l \leq l'$ and $k \leq k'$, $(U, i, l, k)$ leads to a better solution. Threfore,

$$
    V(U, i, l, k) \leq V(U, i, l', k') \text{ if } l \leq l' \land k \leq k'.
$$

The sum of the capacity of the remaining vehicles, $q - l + (m - k) q$, must be greater than or equal to the sum of the weights of the remaining commodities, $\sum_{j \in U} d_j$.
Otherwise, the state does not lead to a solution.
Therefore,

$$
    V(U, i, l, k) = \infty \text{ if } (m - k + 1)q - l < \sum_{j \in U} d_j.
$$

The lowest possible travel cost to visit customer $j$ is $\min_{k \in N \setminus \{ j \}} c_{kj}$.
Because we need to visit all customers in $U$, the total travel cost is at least $\sum_{j \in U} \min_{k \in N \setminus \{ j \}} c_{kj}$. Furthermore, if the current location $i$ is not the depot, we need to visit the depot.
Therefore,

$$
    V(U, i, k, l) \geq \sum_{j \in (U \cup \{ 0 \}) \setminus \{ i \} } \min_{k \in N \setminus \{ j \}} c_{kj}.
$$

Similarly, we need to depart from each customer in $U$ and the current location $i$ if $i$ is not the depot.
Therefore,

$$
    V(U, i, k, l) \geq \sum_{j \in (U \cup \{ i \}) \setminus \{ 0 \} } \min_{k \in N \setminus \{ j \}} c_{jk}.
$$

## Install DIDPPy

In [1]:
!pip install didppy

# Data

In [2]:
# Number of locations
n = 4
# Number of vehicles
m = 2
# Capacity of a vehicle
q = 5
# Weights
d = [0, 2, 3, 3]
# Travel cost
c = [
    [0, 3, 4, 5],
    [3, 0, 5, 4],
    [4, 5, 0, 3],
    [5, 4, 3, 0],
]

## Modeling

In [3]:
import didppy as dp

model = dp.Model()

customer = model.add_object_type(number=n)

# U
unvisited = model.add_set_var(object_type=customer, target=list(range(1, n)))
# i
location = model.add_element_var(object_type=customer, target=0)
# l
load = model.add_int_resource_var(target=0, less_is_better=True)
# k
vehicles = model.add_int_resource_var(target=1, less_is_better=True)

weight = model.add_int_table(d)
distance = model.add_int_table(c)

model.add_base_case([unvisited.is_empty(), location == 0])

for j in range(1, n):
    visit = dp.Transition(
        name="visit {}".format(j),
        cost=distance[location, j] + dp.IntExpr.state_cost(),
        effects=[
            (unvisited, unvisited.remove(j)),
            (location, j),
            (load, load + weight[j]),
        ],
        preconditions=[unvisited.contains(j), load + weight[j] <= q],
    )
    model.add_transition(visit)

for j in range(1, n):
    visit_via_depot = dp.Transition(
        name="visit {} with a new vehicle".format(j),
        cost=distance[location, 0] + distance[0, j] + dp.IntExpr.state_cost(),
        effects=[
            (unvisited, unvisited.remove(j)),
            (location, j),
            (load, weight[j]),
            (vehicles, vehicles + 1),
        ],
        preconditions=[unvisited.contains(j), vehicles < m],
    )
    model.add_transition(visit_via_depot)

return_to_depot = dp.Transition(
    name="return",
    cost=distance[location, 0] + dp.IntExpr.state_cost(),
    effects=[(location, 0)],
    preconditions=[unvisited.is_empty(), location != 0],
)
model.add_transition(return_to_depot)

model.add_state_constr((m - vehicles + 1) * q - load >= weight[unvisited])

min_distance_to = model.add_int_table(
    [min(c[k][j] for k in range(n) if j != k) for j in range(n)]
)
model.add_dual_bound(
    min_distance_to[unvisited] + (location != 0).if_then_else(min_distance_to[0], 0)
)

min_distance_from = model.add_int_table(
    [min(c[j][k] for k in range(n) if j != k) for j in range(n)]
)
model.add_dual_bound(
    min_distance_from[unvisited]
    + (location != 0).if_then_else(min_distance_from[location], 0)
)

## Solving

In [4]:
solver = dp.CABS(model, quiet=True)
solution = solver.search()

print("Transitions to apply:")
print("")

for t in solution.transitions:
    print(t.name)

print("")
print("Cost: {}".format(solution.cost))

Transitions to apply:

visit 1
visit 3
visit 2 with a new vehicle
return

Cost: 20
